<a href="https://colab.research.google.com/github/IndranilPaul15/fuzzy-logic-based-age-classification/blob/main/fuzzy_age.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, IntSlider
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # to avoid matplotlib warnings

# %%
def fuzzy_age_category(age):
    """
    Fuzzy membership calculation for a given age.
    Memberships are capped at < 1 to follow fuzzy set logic.
    """
    cap = 0.99  # Max membership, not exactly 1

    # Baby
    if age <= 3:
        baby = cap
    elif 3 < age < 6:
        baby = (6 - age) / 3 * cap
    else:
        baby = 0

    # Young
    if age <= 25:
        young = cap
    elif 25 < age < 40:
        young = (40 - age) / 15 * cap
    else:
        young = 0

    # Middle-aged
    if age <= 30:
        middle = 0
    elif 30 < age < 45:
        middle = (age - 30) / 15 * cap
    elif 45 <= age < 60:
        middle = (60 - age) / 15 * cap
    else:
        middle = 0

    # Old
    if age <= 50:
        old = 0
    elif 50 < age < 70:
        old = (age - 50) / 20 * cap
    else:
        old = cap

    # Too Old
    if age <= 80:
        too_old = 0
    elif 80 < age < 90:
        too_old = (age - 80) / 10 * cap
    else:
        too_old = cap

    memberships = {
        "Baby": round(baby, 2),
        "Young": round(young, 2),
        "Middle-aged": round(middle, 2),
        "Old": round(old, 2),
        "Too Old": round(too_old, 2)
    }

    # Get max membership label(s)
    max_value = max(memberships.values())
    crisp_labels = [k for k, v in memberships.items() if v == max_value]

    # Mapping for human-friendly names
    mapping = {}
    for label in crisp_labels:
        if label == "Baby":
            mapping[label] = "Baby" if age <= 2 else "Toddler"
        elif label in ("Young", "Middle-aged"):
            mapping[label] = "Adult"
        elif label == "Old":
            mapping[label] = "Senior Citizen"
        elif label == "Too Old":
            mapping[label] = "Super Senior" if age >= 90 else "Elderly"

    human_terms = list(dict.fromkeys(mapping.values()))

    # Build output sentence
    if len(human_terms) == 1:
        sentence = f"The person is a {human_terms[0]}."
    else:
        sentence = f"The person is both {human_terms[0]} and {human_terms[1]}."

    return memberships, crisp_labels, sentence

# %%
def membership_for_category(age, category):
    return fuzzy_age_category(age)[0][category]

# %%
def update_plot(age):
    memberships, labels, sentence = fuzzy_age_category(age)
    categories = list(memberships.keys())

    # Plot membership functions with highlighted point
    ages = np.linspace(0, 100, 500)
    plt.figure(figsize=(10,6))
    for cat in categories:
        plt.plot(ages, [membership_for_category(a, cat) for a in ages], label=cat, linewidth=2)
        plt.scatter(age, memberships[cat], color="red", s=80, zorder=5)
    plt.xlabel("Age")
    plt.ylabel("Membership Degree (< 1)")
    plt.title(f"Fuzzy Membership Functions (Age: {age})")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Heatmap for single age
    plt.figure(figsize=(6,4))
    sns.heatmap(
        np.array([list(memberships.values())]).T,
        annot=True, fmt=".2f", cmap="coolwarm",
        xticklabels=[f"Age {age}"], yticklabels=categories
    )
    plt.title(f"Fuzzy Membership Heatmap (Age: {age})")
    plt.xlabel("Age")
    plt.ylabel("Category")
    plt.show()

    # Classification
    print(sentence)

# %%
# Interactive slider
interact(update_plot, age=IntSlider(min=0, max=100, step=1, value=25, description="Age"));

interactive(children=(IntSlider(value=25, description='Age'), Output()), _dom_classes=('widget-interact',))